<a href="https://colab.research.google.com/github/mboll08/Coursera_Capstone/blob/master/capstone_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import the libraries required 

In [38]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

print('Folium installed')
print('Libraries imported.')


/bin/bash: conda: command not found
/bin/bash: conda: command not found
Folium installed
Libraries imported.


Foresquare credentials

In [39]:
CLIENT_ID = 'EHT1GT25YQ1Z1GLFOIW3135S3ZUON04GZW102HIH54DJOLZS' # your Foursquare ID
CLIENT_SECRET = 'T24ADVGSZCMHWVHZADJLCCPAC00TWC0TW21IFJGZR4UP4AOO' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 150
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: EHT1GT25YQ1Z1GLFOIW3135S3ZUON04GZW102HIH54DJOLZS
CLIENT_SECRET:T24ADVGSZCMHWVHZADJLCCPAC00TWC0TW21IFJGZR4UP4AOO


Define address and location of the place of interest

In [40]:
address = 'St Stephens Green, Dublin'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

53.33798975 -6.259073034461858


GET request and URL

In [41]:
radius =1000

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    latitude, 
    longitude, 
    radius, 
    LIMIT)

Results of venues nearby within 1000 meters of St Stephen's Green

In [42]:
import requests
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5f27e3c98f44ac7ed5db9ff2'},
 'response': {'groups': [{'items': [{'reasons': {'count': 0,
       'items': [{'reasonName': 'globalInteractionReason',
         'summary': 'This spot is popular',
         'type': 'general'}]},
      'referralId': 'e-0-4b2750dcf964a520d78524e3-0',
      'venue': {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/parks_outdoors/park_',
          'suffix': '.png'},
         'id': '4bf58dd8d48988d163941735',
         'name': 'Park',
         'pluralName': 'Parks',
         'primary': True,
         'shortName': 'Park'}],
       'id': '4b2750dcf964a520d78524e3',
       'location': {'address': 'Faiche Stiabhna',
        'cc': 'IE',
        'city': 'Dublin',
        'country': 'Ireland',
        'distance': 18,
        'formattedAddress': ['Faiche Stiabhna',
         'Dublin',
         'Dublin City',
         'Ireland'],
        'labeledLatLngs': [{'label': 'display',
          'lat': 53.33815092558154

Get the venue catergories 

In [43]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Put this data into a dataframe called Dublin venues

In [44]:
venues = results['response']['groups'][0]['items']
    
Dublin_venues = json_normalize(venues) 

filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
Dublin_venues =Dublin_venues.loc[:, filtered_columns]

# filter the category for each row
Dublin_venues['venue.categories'] = Dublin_venues.apply(get_category_type, axis=1)

# clean columns
Dublin_venues.columns = [col.split(".")[-1] for col in Dublin_venues.columns]

Dublin_venues



/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  This is separate from the ipykernel package so we can avoid doing imports until


,name,categories,lat,lng
0,St Stephen's Green,Park,53.338151,-6.259160
1,Hatch & Sons,Café,53.339515,-6.258460
2,Peploe's,French Restaurant,53.339422,-6.258228
3,Peruke & Periwig,Cocktail Bar,53.340086,-6.258542
4,Butlers Chocolate Café,Coffee Shop,53.340066,-6.260777
...,...,...,...,...
95,Wall & Keogh,Tea Room,53.331203,-6.264569
96,Cake Café,Café,53.334488,-6.265880
97,Camden Court Hotel,Hotel,53.332220,-6.263076
98,The Art of Coffee,Café,53.332879,-6.262296


In [45]:
print('{} venues were returned by Foursquare.'.format(Dublin_venues.shape[0]))

100 venues were returned by Foursquare.


See how many of the Dublin venue categories are unique



In [46]:
print('There are {} uniques categories.'.format(len(Dublin_venues['categories'].unique())))

There are 51 uniques categories.


In [47]:
Dublin_venues['categories'].unique()

array(['Park', 'Café', 'French Restaurant', 'Cocktail Bar', 'Coffee Shop',
       'Lounge', 'Ice Cream Shop', 'Gift Shop', 'Burger Joint',
       'Wine Bar', 'Hotel', 'Concert Hall', 'Hotel Bar',
       'Toy / Game Store', 'History Museum', 'Bookstore', 'Donut Shop',
       'Restaurant', 'Tapas Restaurant', 'Italian Restaurant',
       'Chinese Restaurant', 'Beer Bar', 'Pedestrian Plaza', 'Pub',
       'Pizza Place', 'Department Store', 'Vegetarian / Vegan Restaurant',
       'Burrito Place', 'Furniture / Home Store', 'Whisky Bar',
       'Clothing Store', 'Grocery Store', 'Fried Chicken Joint', 'Bar',
       'Shopping Mall', 'Art Museum', 'Steakhouse', 'Outdoor Sculpture',
       'College Library', 'Gym / Fitness Center',
       'Middle Eastern Restaurant', 'Seafood Restaurant',
       'Falafel Restaurant', 'Sandwich Place', 'Plaza', 'Cosmetics Shop',
       'Indian Restaurant', 'Market', 'Bakery', 'Breakfast Spot',
       'Tea Room'], dtype=object)

Check number of venues which are bakeries

In [48]:
Dublin_venues['categories'].value_counts()



Café                             11
Coffee Shop                       8
Pub                               7
Restaurant                        7
Hotel                             6
Burger Joint                      4
Italian Restaurant                3
Park                              3
Cocktail Bar                      2
Falafel Restaurant                2
Bakery                            2
Art Museum                        2
Bar                               2
Wine Bar                          2
Department Store                  2
Ice Cream Shop                    2
Beer Bar                          1
College Library                   1
History Museum                    1
Concert Hall                      1
French Restaurant                 1
Steakhouse                        1
Pizza Place                       1
Toy / Game Store                  1
Gift Shop                         1
Shopping Mall                     1
Middle Eastern Restaurant         1
Grocery Store               

We can see from the count that 2 out of the 100 categories are bakeries 

Visualise location and surrounding bakeries on a map

In [49]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) 

folium.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='St Stephens Green',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

for lat, lng, label in zip(Dublin_venues.lat, Dublin_venues.lng, Dublin_venues.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map

Looking at the map, we can see there are no bakeries in or close to the perimeter of St Stephens Green. This suggests to our client that this would be a suitable location to start up their bakery business as there are no other established competitors in the immediate vicinity. 